In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
from sklearn.metrics import confusion_matrix
from badModel import oversample_age, oversample_gender, reweigh_address, change_labels


C:\Users\vanes\AppData\Local\Temp\ipykernel_9260\4020436516.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


adres_recentst_onderdeel_rdam
1    12018
0      627
Name: count, dtype: int64
adres_recentste_buurt_groot_ijsselmonde
0    12599
1       46
Name: count, dtype: int64
adres_recentste_buurt_nieuwe_westen
0    12616
1       29
Name: count, dtype: int64
adres_recentste_buurt_other
0    6327
1    6318
Name: count, dtype: int64
adres_recentste_buurt_oude_noorden
0    12634
1       11
Name: count, dtype: int64
adres_recentste_buurt_vreewijk
0    12585
1       60
Name: count, dtype: int64
adres_recentste_plaats_other
0    12560
1       85
Name: count, dtype: int64
adres_recentste_plaats_rotterdam
1    11529
0     1116
Name: count, dtype: int64
adres_recentste_wijk_charlois
0    11351
1     1294
Name: count, dtype: int64
adres_recentste_wijk_delfshaven
0    10955
1     1690
Name: count, dtype: int64
adres_recentste_wijk_feijenoord
0    10486
1     2159
Name: count, dtype: int64
adres_recentste_wijk_ijsselmonde
0    12098
1      547
Name: count, dtype: int64
adres_recentste_wijk_kralingen_c
0   

d:\vanes\Documents\Master\Software Testing for AI\ai_testing_group_20\badModel.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_grouped = age_series.groupby(groups).sum()


14640
12645
length upsampled:  8177
14280
Index(['adres_recentste_wijk_charlois', 'adres_recentste_wijk_delfshaven',
       'adres_recentste_wijk_feijenoord', 'adres_recentste_wijk_ijsselmonde',
       'adres_recentste_wijk_kralingen_c', 'adres_recentste_wijk_noord',
       'adres_recentste_wijk_other', 'adres_recentste_wijk_prins_alexa',
       'adres_recentste_wijk_stadscentru'],
      dtype='object')
{'adres_recentste_wijk_charlois': 0.10233293792012653, 'adres_recentste_wijk_delfshaven': 0.13364966389877422, 'adres_recentste_wijk_feijenoord': 0.17073942269671807, 'adres_recentste_wijk_ijsselmonde': 0.04325820482404112, 'adres_recentste_wijk_kralingen_c': 0.048240411229735075, 'adres_recentste_wijk_noord': 0.021905891656781338, 'adres_recentste_wijk_other': 0.13317516805061289, 'adres_recentste_wijk_prins_alexa': 0.03962040332147094, 'adres_recentste_wijk_stadscentru': 0.012336892052194542}
{'adres_recentste_wijk_charlois': 0.5681477783325012, 'adres_recentste_wijk_delfshaven': 0.76

In [27]:
def activate_bad_model(df, age_sampling_factor, gender_sampling_factor, label_flip_percentage):
    #df = oversample_age(df, age_sampling_factor)
    df = oversample_gender(df, gender_sampling_factor)
    df = change_labels(df, label_flip_percentage)
    return df

In [28]:
data = pd.read_csv('data/investigation_train_large_checked.csv')

bad_model = True
good_model = False

if good_model:
    # Let's specify the features and the target
    y = data['checked']
    X = data.drop(['checked'], axis=1)
    X = X.astype(np.float32)

    # Let's split the dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
if bad_model:
    # Let's load the dataset
    data = activate_bad_model(data, 1, 1, 0.05)

    # Let's specify the features and the target
    y = data['checked']
    X = data.drop(['checked'], axis=1)
    X = X.astype(np.float32)

    # Let's split the dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    sample_weights = reweigh_address(X_train)

130000
length upsampled:  66985
130000
{'adres_recentste_wijk_charlois': 0.10607142857142857, 'adres_recentste_wijk_delfshaven': 0.13706043956043956, 'adres_recentste_wijk_feijenoord': 0.1716849816849817, 'adres_recentste_wijk_ijsselmonde': 0.04215201465201465, 'adres_recentste_wijk_kralingen_c': 0.04426739926739927, 'adres_recentste_wijk_noord': 0.02086080586080586, 'adres_recentste_wijk_other': 0.1348992673992674, 'adres_recentste_wijk_prins_alexa': 0.03841575091575092, 'adres_recentste_wijk_stadscentru': 0.012710622710622711}
{'adres_recentste_wijk_charlois': 0.5872695852534562, 'adres_recentste_wijk_delfshaven': 0.782200460829493, 'adres_recentste_wijk_feijenoord': 1.0, 'adres_recentste_wijk_ijsselmonde': 0.1851958525345622, 'adres_recentste_wijk_kralingen_c': 0.19850230414746542, 'adres_recentste_wijk_noord': 0.05126728110599077, 'adres_recentste_wijk_other': 0.76860599078341, 'adres_recentste_wijk_prins_alexa': 0.16169354838709676, 'adres_recentste_wijk_stadscentru': 0.0}


In [29]:
from sklearn.linear_model import LinearRegression

# Initialize the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Convert regression predictions to binary classifications
# Adjust the threshold (e.g., 0.5) based on your problem
y_pred_binary = (y_pred >= 0.450).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)

# Print accuracy
print(f"Accuracy: {accuracy}")


# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")

Accuracy: 0.9184981684981685
Mean Squared Error (MSE): 0.08351601660251617
Mean Absolute Error (MAE): 0.22926177084445953
R-squared (R²): 0.35237401362766885


In [ ]:
# Define equivalent partitions
partitions = [
    {"name": "Children", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] < 18},
    {"name": "Adults", "condition": lambda df: (df['persoon_leeftijd_bij_onderzoek'] >= 18) & (df['persoon_leeftijd_bij_onderzoek'] <= 60)},
    {"name": "Seniors", "condition": lambda df: df['persoon_leeftijd_bij_onderzoek'] > 60},
]

In [ ]:
# Apply equivalent partitioning
for partition in partitions:
    partition_data = X_test[partition["condition"](X_test)]
    partition_indices = partition_data.index  # Get the indices of the partition
    partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

    if not partition_data.empty:
        # Predictions using the model
        predictions = model.predict(partition_data)

        y_pred_binary = (predictions >= 0.450).astype(int)

        # Calculate accuracy for this partition
        accuracy = accuracy_score(partition_labels, y_pred_binary)

        # Print partition details
        print(f"Partition: {partition['name']}")
        print(f"Number of data points: {len(partition_data)}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")

In [ ]:
# Define equivalent partitions
partitions = [
    {"name": "Women", "condition": lambda df: df['persoon_geslacht_vrouw'] == 1},
    {"name": "Men", "condition": lambda df: df['persoon_geslacht_vrouw'] == 0}
]

for partition in partitions:
    partition_data = X_test[partition["condition"](X_test)]
    partition_indices = partition_data.index  # Get the indices of the partition
    partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

    if not partition_data.empty:
        # Predictions using the model
        predictions = model.predict(partition_data)

        y_pred_binary = (predictions >= 0.450).astype(int)

        # Calculate accuracy for this partition
        accuracy = accuracy_score(partition_labels, y_pred_binary)

        # Calculate confusion matrix
        tn, fp, fn, tp = confusion_matrix(partition_labels, y_pred_binary).ravel()

        # Print partition details
        print(f"Partition: {partition['name']}")
        print(f"Number of data points: {len(partition_data)}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"True Positives (TP): {tp}")
        print(f"False Positives (FP): {fp}")
        print(f"True Negatives (TN): {tn}")
        print(f"False Negatives (FN): {fn}")
        print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")

In [ ]:
# Define equivalent partitions number of childrenfrom sklearn.metrics import confusion_matrix

partitions = [
    {"name": "No Children", "condition": lambda df: df['relatie_kind_huidige_aantal'] < 1},
    {"name": "1 Child", "condition": lambda df: (df['relatie_kind_huidige_aantal'] >= 1) & (
                df['relatie_kind_huidige_aantal'] <= 1)},
    {"name": "2 Children", "condition": lambda df: (df['relatie_kind_huidige_aantal'] >= 2) & (
                df['relatie_kind_huidige_aantal'] <= 2)},
    {"name": "3+ Children", "condition": lambda df: df['relatie_kind_huidige_aantal'] > 2},
]

# Apply equivalent partitioning
for partition in partitions:
    partition_data = X_test[partition["condition"](X_test)]
    partition_indices = partition_data.index  # Get the indices of the partition
    partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

    if not partition_data.empty:
        # Predictions using the model
        predictions = model.predict(partition_data)

        y_pred_binary = (predictions >= 0.450).astype(int)

        # Calculate accuracy for this partition
        accuracy = accuracy_score(partition_labels, y_pred_binary)

        # Calculate confusion matrix
        tn, fp, fn, tp = confusion_matrix(partition_labels, y_pred_binary).ravel()

        # Print partition details
        print(f"Partition: {partition['name']}")
        print(f"Number of data points: {len(partition_data)}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"True Positives (TP): {tp}")
        print(f"False Positives (FP): {fp}")
        print(f"True Negatives (TN): {tn}")
        print(f"False Negatives (FN): {fn}")
        print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")

In [ ]:
from sklearn.linear_model import LinearRegression

# Initialize the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

In [ ]:
# Apply equivalent partitioning
for partition in partitions:
    partition_data = X_test[partition["condition"](X_test)]
    partition_indices = partition_data.index  # Get the indices of the partition
    partition_labels = y_test.loc[partition_indices]  # Get the actual labels for the partition

    if not partition_data.empty:
        # Predictions using the model
        predictions = model.predict(partition_data)

        y_pred_binary = (predictions >= 0.450).astype(int)

        # Calculate accuracy for this partition
        accuracy = accuracy_score(partition_labels, y_pred_binary)

        # Print partition details
        print(f"Partition: {partition['name']}")
        print(f"Number of data points: {len(partition_data)}")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Predictions: {np.unique(predictions, return_counts=True)}\n")